# Synapse Labeling Orphans

In [1]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [3]:
dj.config['display.limit'] = 20
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
schema = dj.schema('microns_pinky')

In [ ]:
pinky.OverlayedSpineLabel()

In [7]:


@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> pinky.Synapse
    ---
    (postsynaptic_label)->pinky.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

In [8]:
segment_ids = (pinky.CoarseLabelOrphan - pinky.SegmentExclude).fetch('segment_id')
mesh_fetch = (pinky.Decimation35OrphanStitched & pinky.CompartmentOrphan.proj()).fetch()
label_fetch = (pinky.CoarseLabelOrphan - pinky.SegmentExclude).fetch()

In [ ]:


meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6])
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7])
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

In [ ]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ Old way of inserting without length threshold
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]"""
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

In [ ]:
for syn in synapses_to_exclude:
    syn["criteria_id"] = 0

In [ ]:
print(len(synapses_to_exclude))

In [ ]:
pinky.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)

In [ ]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

# Spine Synapse Labeler

In [4]:
pinky.SynapseCompartmentLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
3,835,2
3,2426,2
3,3848,2
3,5106,2
3,7656,2
3,8128,2
3,8372,2
3,8472,2
3,9439,2
3,9673,2


In [9]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> pinky.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->pinky.LabelKey
    """ # (spine_label)->pinky.LabelKey : tinyint unsigned

In [10]:
mesh_fetch = np.array((pinky.Decimation35OrphanStitched & pinky.CompartmentOrphan.proj()).fetch('segment_id', 'vertices', 'triangles')).T
label_fetch = np.array(pinky.OverlayedSpineLabelOrphan.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 679/679 [02:13<00:00,  5.26it/s]


In [11]:
matched_synapses = dict()
synapses_to_exclude_spine = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ OLD WAY WITHOUTH LENGTH THRESHOLD
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]
    """
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude_spine.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/679 [00:00<?, ?it/s]
0it [00:00, ?it/s]
184it [00:00, 169802.41it/s]
0it [00:00, ?it/s]
  0%|          | 2/679 [00:00<01:01, 10.94it/s]
0it [00:00, ?it/s]
211it [00:00, 235999.51it/s]
0it [00:00, ?it/s]
  1%|          | 4/679 [00:00<00:53, 12.54it/s]
0it [00:00, ?it/s]
178it [00:00, 212581.47it/s]
0it [00:00, ?it/s]
  1%|          | 6/679 [00:00<00:51, 12.99it/s]
0it [00:00, ?it/s]
  1%|          | 7/679 [00:00<01:07, 10.00it/s]
0it [00:00, ?it/s]
61it [00:00, 86407.48it/s]
0it [00:00, ?it/s]
  1%|▏         | 9/679 [00:00<01:04, 10.35it/s]
0it [00:00, ?it/s]
92it [00:00, 120888.46it/s]
0it [00:00, ?it/s]
  2%|▏         | 11/679 [00:01<01:48,  6.18it/s]
0it [00:00, ?it/s]
215it [00:00, 204437.85it/s]
0it [00:00, ?it/s]
  2%|▏         | 13/679 [00:01<01:36,  6.89it/s]
0it [00:00, ?it/s]
150it [00:00, 155114.79it/s]
0it [00:00, ?it/s]
67it [00:00, 96239.17it/s]
0it [00:00, ?it/s]
  2%|▏         | 16/679 [00:01<01:18,  8.49it/s]
0it [00:00, ?it/s]
457it [00:00, 324331.12

In [12]:
#####STILL NEED TO WRITE THE SYNAPSES TO EXCLUDE TO DATABASE

In [13]:
print(len(synapses_to_exclude_spine))


0


In [14]:
for syn in synapses_to_exclude_spine:
    syn["criteria_id"] = 0

In [15]:
synapses_to_exclude_spine

[]

In [16]:
#add to synapse exclude
pinky.SynapseExclude.insert(synapses_to_exclude_spine,skip_duplicates=True)

In [17]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 679/679 [00:00<00:00, 5929.04it/s]


12.88370132446289


## Visualization of Labeled Synapses (no labeled Spines yet)

In [ ]:
# pinky.LabelKey()

In [ ]:
# pinky.ProofreadLabel & 'segment_id=648518346341393609'

In [ ]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'ignore blue'), ( 3, 'Basal', 'ignore yellow'),
#                ( 4, 'Oblique', 'ignore green'), ( 5, 'Soma', 'ignore red'),
#                ( 6, 'Axon-Soma', 'ignore aqua'), ( 7, 'Axon-Dendr', 'ignore off blue'),
#                ( 8, 'Dendrite', 'ignore purple'), ( 9, 'Distal', 'ignore pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'ignore tan'),
#                (12, 'Cilia', 'ignore orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'ignore green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()